In [26]:
###############################################
#  1. CA Infrastructure & PQC Library Setup   #
###############################################

# This cell outlines the environment setup. In a real scenario, 
# you'd have a Docker/Kubernetes environment or a specialized server.


# Also, if you are using a custom OpenSSL with PQC, ensure it’s on your PATH.
import sys
import os

print("Python version:", sys.version)
print("Current working directory:", os.getcwd())



Python version: 3.12.7 (main, Oct  1 2024, 02:05:46) [Clang 16.0.0 (clang-1600.0.26.3)]
Current working directory: /Users/abhisekjha/MyFolder/Github_Projects/NextGenSecureMessaging


In [17]:
import os
import sys

# Add paths
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.join(os.getcwd(), 'dilithium/src'))
sys.path.append(os.path.join(os.getcwd(), 'pyky'))



from dilithium_py.dilithium import Dilithium2

# Generate Dilithium key pair
try:
    dilithium_keypair = Dilithium2.keygen()
    dilithium_public_key, dilithium_private_key = dilithium_keypair

    print("Dilithium2 public key length:", len(dilithium_public_key))
    print("Dilithium2 private key length:", len(dilithium_private_key))
except Exception as e:
    print(f"Dilithium key generation error: {e}")
    sys.exit(1)



from src.key_management import generate_kyber_keypair

# Generate Kyber key pair
kyber_private_key, kyber_public_key = generate_kyber_keypair()
print("Kyber512 private key length:", len(kyber_private_key))
print("Kyber512 public key length:", len(kyber_public_key))


Dilithium2 public key length: 1312
Dilithium2 private key length: 2528
Kyber512 private key length: 3168
Kyber512 public key length: 1568


Explanation

1. We use `dilithium` and `kyber` custom repo to generate keys.
2. Dilithium is used for signatures (like X.509 certificates).
3. Kyber is a Key Encapsulation Mechanism (KEM) used for key exchange in TLS, IPsec, or AKA.


In [45]:
#################################################################
# 3. Mock CA: Issuing PQC Certificates (Dilithium-signed X.509) #
#################################################################

# In production, you'd integrate with a real PKI (e.g., HashiCorp Vault, 
# EJBCA, or an OpenSSL-based CA). For illustration, let’s do a simplified 
# approach to show how you might sign a certificate with Dilithium.

# We'll create a minimal X.509-like structure in Python and "sign" it 
# with the Dilithium private key. 
# In practice, you'd use an OpenSSL or libpki approach with PQC patches.

from datetime import datetime, timedelta
import base64
import json
import sys
import os

# Add paths for custom modules
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.join(os.getcwd(), 'dilithium/src'))
sys.path.append(os.path.join(os.getcwd(), 'pyky'))

from dilithium_py.dilithium import Dilithium2
from src.key_management import generate_kyber_keypair


# Generate Dilithium key pair
try:
    dilithium_keypair = Dilithium2.keygen()
    dilithium_public_key, dilithium_private_key = dilithium_keypair

    print("Dilithium2 public key length:", len(dilithium_public_key))
    print("Dilithium2 private key length:", len(dilithium_private_key))
except Exception as e:
    print(f"Dilithium key generation error: {e}")
    sys.exit(1)

# Generate Kyber key pair
try:
    kyber_private_key, kyber_public_key = generate_kyber_keypair()
    print("Kyber512 private key length:", len(kyber_private_key))
    print("Kyber512 public key length:", len(kyber_public_key))
except Exception as e:
    print(f"Kyber key generation error: {e}")
    sys.exit(1)


def create_mock_certificate(subject_name: str, pub_key: bytes, issuer_name: str, validity_days=30):
    """
    Create a mock certificate structure in JSON form.
    This is not a real X.509, but a stand-in to show the process.
    """
    cert_data = {
        "subject": subject_name,
        "issuer": issuer_name,
        "valid_from": datetime.utcnow().isoformat() + "Z",
        "valid_to": (datetime.utcnow() + timedelta(days=validity_days)).isoformat() + "Z",
        "pqc_public_key": base64.b64encode(pub_key).decode('utf-8'),  # store the PQC public key
    }
    return cert_data


def sign_certificate(cert_data: dict, private_key: bytes, signature_scheme: str="Dilithium2"):
    """
    Sign the certificate data with the Dilithium private key
    using the dilithium_py package. We'll store the signature in the cert.
    """
    try:
        message_bytes = json.dumps(cert_data, sort_keys=True).encode('utf-8')
        signature = Dilithium2.sign(private_key, message_bytes)
        cert_data["pqc_signature"] = base64.b64encode(signature).decode('utf-8')
        return cert_data
    except Exception as e:
        print(f"Error signing certificate: {e}")
        sys.exit(1)

# Convert the Kyber public key to bytes if it's a list
# Ensure the Kyber public key is a bytes-like object
if isinstance(kyber_public_key, list):
    # Normalize values to the range 0-255
    kyber_public_key = bytes([x % 256 for x in kyber_public_key])

    
# 3.1 Create a "Root CA" certificate using Dilithium
root_subject = "CN=MyPQC-RootCA"
root_cert_struct = create_mock_certificate(
    subject_name=root_subject,
    pub_key=dilithium_public_key,
    issuer_name=root_subject,  # self-signed
    validity_days=365
)
root_cert_signed = sign_certificate(root_cert_struct, dilithium_private_key, "Dilithium2")

# 3.2 Create an "End-Entity" certificate signed by the root
# 3.2 Create an "End-Entity" certificate signed by the root
end_entity_subject = "CN=MyServer"
end_entity_cert_struct = create_mock_certificate(
    subject_name=end_entity_subject,
    pub_key=kyber_public_key,  # Ensure this is bytes
    issuer_name=root_subject,
    validity_days=90
)
end_entity_cert_signed = sign_certificate(end_entity_cert_struct, dilithium_private_key, "Dilithium2")
print("Root CA Certificate:\n", json.dumps(root_cert_signed, indent=2))
print("\nEnd-Entity Certificate:\n", json.dumps(end_entity_cert_signed, indent=2))


Dilithium2 public key length: 1312
Dilithium2 private key length: 2528
Kyber512 private key length: 3168
Kyber512 public key length: 1568
Root CA Certificate:
 {
  "subject": "CN=MyPQC-RootCA",
  "issuer": "CN=MyPQC-RootCA",
  "valid_from": "2024-12-27T02:08:50.895442Z",
  "valid_to": "2025-12-27T02:08:50.895487Z",
  "pqc_public_key": "Ru0T/DSZmFzm47Uf+qibHGTrQBK9WGyvJf/AYSLtgaMh8eu918A2rIDCnjQzt2SU0ggWOx96w3LM8t+cCt8WF0jQmGcYNpA9ZTvZyVg+l93G4Z/b0fY292yvsbQ5PuwDlL2oDRJKudAJbwvjUXcwpN66XQ/KbvHwwXYv8AYQjdu1UR/R4w/ZBik2q23f4ZzmbQ7oFVphAAPYz9MJsKTTQ2caNC7Vnovy4wL7/SpR3jcKW7qViS51zlDiVpGFDkqgvwT5qY+SVF42kytoaaiP8kF7F6E5nUIl3tp7lnkDCI4/Qcz7SSvodcIhFUVJT4BLAaiYOQvh2rhwobF9O44qONJcwGZBw9tsawk9X1NAw6Nmp+yqd9R5FgADQooQQyGsgKL4bpG8ekaWmsoTLW51PAQ6+5OW52ealk7WHoIdD8qWNC2fL6FL15R/nktQxGv9Dml1SILv3fKDM8Kj5CxWen7rnb+NRaiAo5byahQgKbSWClqncCJNJqW9GNQqLZLOaATt7VvaUdu7zF9My4WqsJkOHEjmLY8msa2KGR+N6BuxSLQ6iqlup77i86d7Q6c0BBEAOaid7QwGH6PsyUu88DC9NcFqG5irvBQz8a5wrRabG8pyfsA2t5w4qhGie3P8AsZP/gIzvGclDnvtLZED0c

/var/folders/tr/m7nnfyd94_jfxmdwqvwq7hg40000gn/T/ipykernel_76197/2124973119.py:57: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "valid_from": datetime.utcnow().isoformat() + "Z",
/var/folders/tr/m7nnfyd94_jfxmdwqvwq7hg40000gn/T/ipykernel_76197/2124973119.py:58: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "valid_to": (datetime.utcnow() + timedelta(days=validity_days)).isoformat() + "Z",


In [57]:
#########################################################
# 4. Verifying the PQC Certificate (Dilithium Signature)#
#########################################################

def verify_pqc_certificate(cert_data: dict, root_pub_key: bytes, signature_scheme: str = "Dilithium2"):
    """
    Verify the 'pqc_signature' field using the root's public key.
    """
    try:
        # Extract the signature and remove it from the certificate data for verification
        signature_b64 = cert_data.pop("pqc_signature", None)
        if not signature_b64:
            raise ValueError("No signature found in certificate data.")

        # Reconstruct the message (certificate data) without the signature
        message_bytes = json.dumps(cert_data, sort_keys=True).encode('utf-8')
        signature = base64.b64decode(signature_b64)

        # Perform verification using Dilithium2
        is_valid = Dilithium2.verify(root_pub_key, message_bytes, signature)
        if is_valid:
            print("Certificate is VALID under PQC signature:", signature_scheme)
        else:
            print("Certificate is INVALID: Signature mismatch.")
    except Exception as e:
        print("Verification failed due to an error:", e)
    finally:
        # Restore the signature in the certificate data for further use
        cert_data["pqc_signature"] = signature_b64

# Verify the root certificate (self-signed)
print("Verifying the Root CA certificate:")
verify_pqc_certificate(root_cert_signed, dilithium_public_key, "Dilithium2")

# Verify the end-entity certificate with the root CA's public key
print("\nVerifying the End-Entity certificate:")
verify_pqc_certificate(end_entity_cert_signed, dilithium_public_key, "Dilithium2")


NameError: name 'Root' is not defined